In [1]:
from langchain.tools.retriever import create_retriever_tool

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_milvus import Milvus
import os

from tqdm.auto import tqdm

In [2]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="dotenv.txt");

In [3]:
MILVUS_HOST = os.getenv('MILVUS_HOST',"vectordb-milvus.milvus.svc.cluster.local")
MILVUS_PORT = 19530
MILVUS_USERNAME = os.getenv('MILVUS_USERNAME')
MILVUS_PASSWORD = os.getenv('MILVUS_PASSWORD')
MILVUS_COLLECTION = "demo_collection"

In [7]:
MILVUS_HOST

'vectordb-milvus.milvus.svc.cluster.local'

In [6]:
model_kwargs = {'trust_remote_code': True}
embeddings = HuggingFaceEmbeddings(
    #model_name="nomic-ai/nomic-embed-text-v1",
    model_kwargs=model_kwargs,
    show_progress=False
)

In [9]:
store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": f"http://{MILVUS_HOST}:{MILVUS_PORT}", "user": MILVUS_USERNAME, "password": MILVUS_PASSWORD},
    collection_name=MILVUS_COLLECTION,
    metadata_field="metadata",
    text_field="page_content",
    drop_old=False
    )

In [10]:
retriever = store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 4}
            )

In [5]:
rag_tool_description = """
Use this tool when seaching for documentation information about Red Hat OpenShift AI
"""

In [6]:
tool_retriever = create_retriever_tool(retriever, "openshift_ai_documentation_search_tool", rag_tool_description)

In [7]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"